# HealTac 2024 Tutorial
## PubMed abstract information extraction

- Yunsoo Kim (yunsoo.kim.23@ucl.ac.uk), Jinge Wu (jinge.wu.20@ucl.ac.uk), Honghan Wu (honghan.wu@ucl.ac.uk)

<a target="_blank" href="https://colab.research.google.com/github/knowlab/healtac2024_tutorial/blob/main/pubmed_IE.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Installing dependencies

In [ ]:
!nvidia-smi

Sun Jun  9 22:15:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

In [ ]:
import requests
from bs4 import BeautifulSoup

We are going to use entrez eutil to get abstract

In [ ]:
def get_pmids(term, maxCount):
    if int(maxCount)<10000:
        r=requests.get(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={term}&RetMax={maxCount}')
        soup=BeautifulSoup(r.text)
        try:
            return [i.get_text(strip=True) for i in soup.find_all('id')]
        except:
            return []
    else:
        pmid_list=[]
        for i in range(0, int(maxCount), 9999):
            r=requests.get(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={term}&RetMax=9999&retstart={i}')
            soup=BeautifulSoup(r.text)
            try:
                pmid_list=pmid_list+[i.get_text(strip=True) for i in soup.find_all('id')]
            except:
                return []
        return pmid_list

In [ ]:
def get_pubmed_abstracts(pmid):
    r=requests.get(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmid}')
    soup = BeautifulSoup(r.text)
    try:
        return '\n'.join([i.get_text(strip=True) for i in soup.find_all("abstracttext")])
    except:
        return ""

In [ ]:
pmids=get_pmids('SCN5A', 1)
print(pmids)

['38849547']


<ipython-input-4-121ac26679b8>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup=BeautifulSoup(r.text)


In [ ]:
get_pubmed_abstracts('38849547')

<ipython-input-5-5c59f673bccf>:3: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(r.text)


'Sudden infant death syndrome (SIDS) is still the leading cause of death for newborns in developed countries. The pathophysiological mechanisms have not been fully clarified, but in some of SIDS cases variants of genes associated with inherited cardiac conditions are found. In this study, an analysis of SCD-related genes was performed to determine the prevalence of rare pathogenic (P) or likely pathogenic (LP) variants that could provide an unambiguous explanation for the fatal event. A cohort of 76 SIDS cases underwent Next-Generation Sequencing (NGS) analysis with a custom panel of SCD-related genes. Rare variants were classified according to the guidelines provided by the American College of Medical Genetics and Genomics (ACMG) and the specifications of the ClinGen association. Post-mortem genetic testing identified 50 (65.8%) carriers of at least one variant in SCD genes. 104 rare genetic variants were found, 65.4% in genes encoding structural proteins. Only 4 out of 76 cases (5.3%

In [ ]:
data={'pmid': '15485686',
 'text': 'A novel SCN5A mutation manifests as a malignant form of long QT syndrome with perinatal onset of tachycardia/bradycardia.\nOBJECTIVE: Congenital long QT syndrome (LQTS) with in utero onset of the rhythm disturbances is associated with a poor prognosis. In this study we investigated a newborn patient with fetal bradycardia, 2:1 atrioventricular block and ventricular tachycardia soon after birth. METHODS: Mutational analysis and DNA sequencing were conducted in a newborn. The 2:1 atrioventricular block improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine, which also controlled the ventricular tachycardia. RESULTS: A novel, spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel, with a G-->A substitution at codon 1763, which changed a valine (GTG) to a methionine (ATG). The proband was heterozygous but the mutation was absent in the parents and the sister. Expression of this mutant channel in tsA201 mammalian cells by site-directed mutagenesis revealed a persistent tetrodotoxin-sensitive but lidocaine-resistant current that was associated with a positive shift of the steady-state inactivation curve, steeper activation curve and faster recovery from inactivation. We also found a similar electrophysiological profile for the neighboring V1764M mutant. But, the other neighboring I1762A mutant had no persistent current and was still associated with a positive shift of inactivation. CONCLUSIONS: These findings suggest that the Na(v)1.5/V1763M channel dysfunction and possible neighboring mutants contribute to a persistent inward current due to altered inactivation kinetics and clinically congenital LQTS with perinatal onset of arrhythmias that responded to lidocaine and mexiletine.',
 'entities': ['sodium',
  'valine (GTG) to a methionine (ATG)',
  'atrioventricular block',
  'tachycardia',
  'long QT syndrome',
  'bradycardia',
  'mexiletine',
  'Na(v)1.5',
  'SCN5A',
  'LQTS',
  'G-->A substitution at codon 1763',
  'V1763M',
  'lidocaine',
  'V1764M',
  'arrhythmias',
  'I1762A',
  'patient',
  'tetrodotoxin',
  'tsA201',
  'ventricular tachycardia']}

In [ ]:
text=data['text']

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        'bluesky333/medphi2',
        device_map="auto",
        trust_remote_code=True,
    ).eval()

model.generation_config = GenerationConfig.from_pretrained(
    'bluesky333/medphi2', trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/bluesky333/medphi2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'bluesky333/medphi2'
)

tokenizer_config.json:   0%|          | 0.00/7.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def get_response(question):
    inputs = tokenizer(question, return_tensors="pt", return_attention_mask=False).to(DEVICE)
    response = model.generate(**inputs,
                         eos_token_id=tokenizer.eos_token_id,  # End of sequence token
                         pad_token_id=tokenizer.eos_token_id,  # Pad token
                         max_new_tokens=1024,
                        )
    output = tokenizer.decode(response.squeeze()[len(inputs['input_ids'][0]):], skip_special_tokens=True)
    return output

In [ ]:
data['entities']

['sodium',
 'valine (GTG) to a methionine (ATG)',
 'atrioventricular block',
 'tachycardia',
 'long QT syndrome',
 'bradycardia',
 'mexiletine',
 'Na(v)1.5',
 'SCN5A',
 'LQTS',
 'G-->A substitution at codon 1763',
 'V1763M',
 'lidocaine',
 'V1764M',
 'arrhythmias',
 'I1762A',
 'patient',
 'tetrodotoxin',
 'tsA201',
 'ventricular tachycardia']

In [ ]:
get_response("Extract biomedical named entities. \n"+text)

' The SCN5A mutation identified in this study is the first to be associated with a malignant form of LQTS with perinatal onset of tachycardia/bradycardia.The biomedical named entities that can be extracted from the given text are: SCN5A mutation, long QT syndrome, perinatal onset, tachycardia, bradycardia, in utero onset, 2:1 atrioventricular block, ventricular tachycardia, intravenous lidocaine infusion, mexiletine, Na(v)1.5 channel, G-->A substitution, codon 1763, valine, methionine, tsA201 mammalian cells, site-directed mutagenesis, tetrodotoxin-sensitive, lidocaine-resistant current, positive shift of the steady-state inactivation curve, steeper activation curve, faster recovery from inactivation, neighboring V1764M mutant, and neighboring I1762A mutant.'

In [ ]:
print(get_response("Extract biomedical named entities and map them with their types. \n"+text))

The biomedical named entities in the given text are: SCN5A mutation, long QT syndrome, perinatal onset, tachycardia, bradycardia, in utero onset, 2:1 atrioventricular block, ventricular tachycardia, intravenous lidocaine infusion, mexiletine, Na(v)1.5 channel, transmembrane segment 6, domain IV, G-->A substitution, codon 1763, valine, methionine, tsA201 mammalian cells, site-directed mutagenesis, tetrodotoxin-sensitive, lidocaine-resistant current, positive shift of the steady-state inactivation curve, steeper activation curve, faster recovery from inactivation, V1764M mutant, I1762A mutant. 

The types of the biomedical named entities are: 
- SCN5A mutation: genetic mutation 
- Long QT syndrome: medical condition 
- Perinatal onset: time of onset 
- Tachycardia: medical condition 
- Bradycardia: medical condition 
- 2:1 atrioventricular block: cardiac conduction disorder 
- Ventricular tachycardia: medical condition 
- Intravenous lidocaine infusion: medical treatment 
- Mexiletine: m

In [ ]:
print(get_response("Extract biomedical named entities and map them with their types. Give me the result in a table.  \n"+text))

The following is a table of the biomedical named entities and their types:

| Named Entity | Type |
| --- | --- |
| SCN5A mutation | Genetic mutation |
| Long QT syndrome | Genetic disorder |
| Perinatal onset | Medical condition |
| Tachycardia/bradycardia | Medical condition |
| Atrioventricular block | Cardiac arrhythmia |
| Ventricular tachycardia | Cardiac arrhythmia |
| Na(v)1.5 channel | Cardiac sodium channel |
| G-->A substitution | Genetic mutation |
| Valine (GTG) | Amino acid |
| Methionine (ATG) | Amino acid |
| Lidocaine | Medication |
| Mexiletine | Medication |
| TsA201 cells | Cell line |
| V1764M mutant | Genetic mutation |
| I1762A mutant | Genetic mutation |

Note: The reason why the instruction is unanswerable is that the given text does not provide information on the specific types of the medical conditions and cardiac arrhythmias mentioned.


In context learning

In [ ]:
print(get_response("Extract biomedical named entities and map them with their types. Also, do relation extraction. Give me the result in a table.  \n"+text))

The following is a table of the extracted biomedical named entities and their types, as well as the relation extraction results:

| Named Entity | Type | Relation |
| --- | --- | --- |
| SCN5A mutation | Genetic mutation | Associated with malignant form of long QT syndrome |
| Long QT syndrome | Genetic disorder | Associated with poor prognosis and perinatal onset of arrhythmias |
| Fetal bradycardia | Cardiac arrhythmia | Present in newborn patient with 2:1 atrioventricular block and ventricular tachycardia |
| Atrioventricular block | Cardiac arrhythmia | Improved to 1:1 conduction after intravenous lidocaine infusion or a high dose of mexiletine |
| Ventricular tachycardia | Cardiac arrhythmia | Controlled by intravenous lidocaine infusion or a high dose of mexiletine |
| Na(v)1.5 channel | Cardiac sodium channel | Mutated in the transmembrane segment 6 of domain IV, with a G-->A substitution at codon 1763 |
| Lidocaine | Medication | Effective in controlling the ventricular tachyca

In [ ]:
print(get_response("Extract biomedical named entities and map them with their types. Also, do relation extraction. The relationship format should be Gene A <-> Disease B if Gene A has an association with Disease B. Give me the result in a table.  \n"+text))

The following biomedical named entities were extracted and mapped with their types:
- SCN5A mutation
- Long QT syndrome
- Tachycardia
- Bradycardia
- Atrioventricular block
- Ventricular tachycardia
- Na(v)1.5 channel
- G-->A substitution
- Valine (GTG)
- Methionine (ATG)
- Lidocaine
- Mexiletine
- TsA201 mammalian cells
- V1764M mutant
- I1762A mutant

The following relations were extracted:
- SCN5A mutation <-> Long QT syndrome
- Long QT syndrome <-> Tachycardia
- Long QT syndrome <-> Bradycardia
- Long QT syndrome <-> Atrioventricular block
- Long QT syndrome <-> Ventricular tachycardia
- Na(v)1.5 channel mutation <-> Na(v)1.5/V1763M channel dysfunction
- Na(v)1.5/V1763M channel dysfunction <-> Persistent inward current
- Na(v)1.5/V1763M channel dysfunction <-> Congenital LQTS with perinatal onset of arrhythmias
- Lidocaine and mexiletine <-> Control of ventricular tachycardia

The result in a table format is as follows:

| Biomedical Named Entity | Type |
|---------------------|---

Chain of Thought

In [ ]:
get_pubmed_abstracts('38849547')

<ipython-input-5-5c59f673bccf>:3: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(r.text)


In [ ]:
text0='Sudden infant death syndrome (SIDS) is still the leading cause of death for newborns in developed countries. The pathophysiological mechanisms have not been fully clarified, but in some of SIDS cases variants of genes associated with inherited cardiac conditions are found. In this study, an analysis of SCD-related genes was performed to determine the prevalence of rare pathogenic (P) or likely pathogenic (LP) variants that could provide an unambiguous explanation for the fatal event. A cohort of 76 SIDS cases underwent Next-Generation Sequencing (NGS) analysis with a custom panel of SCD-related genes. Rare variants were classified according to the guidelines provided by the American College of Medical Genetics and Genomics (ACMG) and the specifications of the ClinGen association. Post-mortem genetic testing identified 50 (65.8%) carriers of at least one variant in SCD genes. 104 rare genetic variants were found, 65.4% in genes encoding structural proteins. Only 4 out of 76 cases (5.3%) hosted at least a P or LP variant found in genes with structural or structural/arrhythmogenic functions (SLC22A5, SCN5A, MYL3and TTN). 99 variants were classified as of uncertain significance (VUS)'

In [ ]:
get_response("Calculate the chi-square p-value of the difference in the distribution of variants between gene groups by function. \n"+text0)

'The chi-square p-value of the difference in the distribution of variants between gene groups by function is not provided in the given discharge summary. Therefore, it is unanswerable with the given information.'

In [ ]:
print(get_response("Calculate the chi-square p-value of the difference in the distribution of variants between gene groups by function. Show me the step by step calculation. \n"+text0))

The chi-square p-value of the difference in the distribution of variants between gene groups by function can be calculated using the following formula:

χ² = ∑ [(O - E)² / E]

where O is the observed frequency and E is the expected frequency.

In this case, we have 4 cases with variants in genes with structural or structural/arrhythmogenic functions (SLC22A5, SCN5A, MYL3and TTN), and 76 total cases. Therefore, the expected frequency is:

E = (4/76) x 100 = 5.3%

The observed frequency is:

O = 4

Plugging these values into the formula, we get:

χ² = [(4-5.3)² / 5.3] = 1.42

Using a chi-square distribution table with 1 degree of freedom, the p-value is approximately 0.17.

Therefore, the chi-square p-value of the difference in the distribution of variants between gene groups by function is 0.17.
